# Objetivo

Aqui vamos atuar em cima das bases mensais para calcular conceitos e avaliações fundamentalistas. São elas:
- Graham atualizado por setor
- Flag - Acima ou abaixo do preço de graham ajustado?
- Flag - Acima ou abaixo do `DY %` médio do setor?
- Flag - Acima de 10% de DY?

## Bibliotecas

In [59]:
# Geral
import warnings
import numpy as np
import pandas as pd

# Plots
import seaborn as sns
import plotly.express as px
import matplotlib.pyplot as plt

# Remover warnings chatos do pandas
warnings.filterwarnings("ignore")

## Calcular Preço de Graham Ajustado

In [78]:
def calcular_preco_graham_setor(pl, pvp, lpa, vpa):
    # Calcular valor
    val = pl * pvp * lpa * vpa
    if val > 0:
        return round(np.sqrt(val), 2)
    else:
        return np.nan

# Execução

In [16]:
# Carregar base(s)
df = pd.read_csv('dados/2_TRA/fundamentus_2023-11-02.csv')

**Agregados por setor necessários**

In [17]:
# Gerar agregados
df_agg = df[['SETOR', 'P_POR_L', 'P_POR_VP', 'DIV_YIELD']].groupby('SETOR').median().reset_index()

In [18]:
# Modificar colunas
df_agg.columns = [c if c == 'SETOR' else f'AGG_{c}' for c in df_agg.columns]

In [19]:
# Unificar informações na base
df = pd.merge(df, df_agg, on='SETOR', how='left')

In [20]:
# Calcular cotação de graham ajustada ao setor
df['COTACAO_GRAHAM_SETOR'] = df.apply(lambda x: calcular_preco_graham_setor(x['AGG_P_POR_L'],
                                                                            x['AGG_P_POR_VP'],
                                                                            x['LPA'],
                                                                            x['VPA']), axis=1)

In [47]:
# Criar razão - Acima indica um valor de cotação abaixo do que deveria
df['RAZAO_COTACAO_GRAHAM'] = (df['COTACAO_GRAHAM_SETOR']/df['COTACAO']).round(2)

**Criar Flags**

In [21]:
df['FLAG_MENOR_GRAHAM_SETOR'] = (df['COTACAO'] < df['COTACAO_GRAHAM_SETOR']).astype(int)

In [22]:
df['FLAG_DY_ACIMA_10'] = (df['DIV_YIELD'] >= 10).astype(int)

In [23]:
df['FLAG_DY_ACIMA_DY_SETOR'] = (df['DIV_YIELD'] >= df['AGG_DIV_YIELD']).astype(int)

In [67]:
df['FLAG_MENOR_GRAHAM_ACIMA_DY_SETOR'] = (df['FLAG_DY_ACIMA_DY_SETOR'] == 1) & (df['FLAG_DY_ACIMA_DY_SETOR'] == 1)

**Avaliação de Carteira**

In [70]:
# Definir papeis atuais a partir da carteira e seus setores
papeis_atuais = ['BBAS3','ITSA3','EGIE3','TAEE4','PETR4','PRIO3','VALE3','TUPY3']
setores_atuais = df.loc[df['PAPEL'].isin(papeis_atuais), 'SETOR'].unique().tolist()

In [71]:
# Criar dataframe de análise
df_aval = df.loc[df['SETOR'].isin(setores_atuais)] 

In [72]:
# Flag para apontar papeis de sua carteira
df_aval['CARTEIRA'] = df_aval['PAPEL'].isin(papeis_atuais).astype(int).copy()

In [73]:
# Definir colunas de analise
cols = ['PAPEL','SETOR','COTACAO','COTACAO_GRAHAM_SETOR', 'RAZAO_COTACAO_GRAHAM', 'DIV_YIELD','AGG_DIV_YIELD', 'P_POR_L']
cols += ['OSCILACOES_%_2023', 'LPA', 'VPA', 'MARG_EBIT', 'MARG_LIQUIDA']
cols += [c for c in df.columns if 'FLAG' in c]

In [74]:
# Visualizar
df_aval.loc[df_aval['CARTEIRA'] == 1, cols]

,PAPEL,SETOR,COTACAO,COTACAO_GRAHAM_SETOR,RAZAO_COTACAO_GRAHAM,DIV_YIELD,AGG_DIV_YIELD,P_POR_L,OSCILACOES_%_2023,LPA,VPA,MARG_EBIT,MARG_LIQUIDA,FLAG_MENOR_GRAHAM_SETOR,FLAG_DY_ACIMA_10,FLAG_DY_ACIMA_DY_SETOR,FLAG_MENOR_GRAHAM_ACIMA_DY_SETOR
155,TAEE4,Energia Elétrica,11.27,9.17,0.81,10.0,6.80,9.97,2.98,1.13,6.47,63.1,49.8,0,1,1,True
157,EGIE3,Energia Elétrica,40.62,21.60,0.53,8.7,6.80,10.23,14.38,3.97,10.22,46.6,28.5,0,0,1,True
210,BBAS3,Intermediários Financeiros,49.34,68.82,1.39,9.3,6.55,4.27,52.02,11.55,54.86,NaN,0.0,1,0,1,True
218,ITSA3,Intermediários Financeiros,8.82,9.01,1.02,7.8,6.55,6.45,4.08,1.37,7.92,14.1,175.5,1,0,1,True
283,TUPY3,Material de Transporte,24.91,22.39,0.90,3.0,2.95,7.93,-5.31,3.14,21.56,8.2,4.2,0,0,1,True
291,VALE3,Mineração,62.68,50.97,0.81,6.5,12.70,5.00,-26.08,12.54,41.20,37.8,27.8,0,0,0,False
303,PETR4,"Petróleo, Gás e Biocombustíveis",37.85,49.10,1.30,24.4,1.35,3.16,90.66,11.99,28.42,45.0,27.0,1,1,1,True
306,PRIO3,"Petróleo, Gás e Biocombustíveis",49.70,18.98,0.38,0.0,1.35,11.85,33.57,4.20,12.12,57.3,46.0,0,0,0,False


In [77]:
df_viz[cols]

,PAPEL,SETOR,COTACAO,COTACAO_GRAHAM_SETOR,RAZAO_COTACAO_GRAHAM,DIV_YIELD,AGG_DIV_YIELD,P_POR_L,OSCILACOES_%_2023,LPA,VPA,MARG_EBIT,MARG_LIQUIDA,FLAG_MENOR_GRAHAM_SETOR,FLAG_DY_ACIMA_10,FLAG_DY_ACIMA_DY_SETOR,FLAG_MENOR_GRAHAM_ACIMA_DY_SETOR
206,MERC4,Intermediários Financeiros,7.16,NaN,NaN,6.3,6.55,-35.86,1.10,-0.20,13.81,NaN,0.0,0,0,0,False
207,BRBI11,Intermediários Financeiros,11.65,NaN,NaN,7.2,6.55,0.00,2.87,0.00,0.00,NaN,NaN,0,0,1,True
208,BNBR3,Intermediários Financeiros,93.06,148.53,1.60,6.7,6.55,3.60,32.07,25.85,114.19,NaN,0.0,1,0,1,True
209,BAZA3,Intermediários Financeiros,86.89,127.39,1.47,6.4,6.55,3.90,85.77,22.28,97.45,NaN,0.0,1,0,0,False
210,BBAS3,Intermediários Financeiros,49.34,68.82,1.39,9.3,6.55,4.27,52.02,11.55,54.86,NaN,0.0,1,0,1,True
211,BMEB4,Intermediários Financeiros,11.64,16.03,1.38,6.9,6.55,4.46,23.02,2.61,13.18,NaN,0.0,1,0,1,True
212,ABCB4,Intermediários Financeiros,19.35,25.30,1.31,6.2,6.55,5.42,5.04,3.57,23.98,NaN,0.0,1,0,0,False
213,PINE4,Intermediários Financeiros,3.53,4.62,1.31,4.6,6.55,6.18,127.05,0.57,5.02,NaN,0.0,1,0,0,False
214,BGIP4,Intermediários Financeiros,22.36,31.05,1.39,15.3,6.55,6.13,46.85,3.65,35.35,NaN,0.0,1,1,1,True
215,BMEB3,Intermediários Financeiros,13.47,16.03,1.19,5.4,6.55,5.16,41.09,2.61,13.18,NaN,0.0,1,0,0,False


In [82]:
# Gráfico Setorial 1 - P_POR_L (Anos de retorno de investimento via lucro) x DIV_YIELD (%)
# Definir dataframe para visualizar
cond = (df_aval['SETOR'] == setores_atuais[2]) & (df_aval['DIV_YIELD'] < 1000) & (df_aval['COTACAO_GRAHAM_SETOR'].notnull())
df_viz = df_aval[cond].copy()

fig = px.scatter(data_frame=df_viz, x='P_POR_VP', y='DIV_YIELD',
                 hover_data=['AGG_DIV_YIELD','PAPEL','COTACAO_GRAHAM_SETOR','COTACAO'], color='FLAG_MENOR_GRAHAM_ACIMA_DY_SETOR')
fig.show()

In [ ]:
# Gráfico Setorial 1 - P_POR_L (Anos de retorno de investimento via lucro) x DIV_YIELD (%)
# Definir dataframe para visualizar
cond = (df_aval['SETOR'] == setores_atuais[1]) & (df_aval['DIV_YIELD'] < 1000) & (df_aval['COTACAO_GRAHAM_SETOR'].notnull())
df_viz = df_aval[cond].copy()

fig = px.scatter(data_frame=df_viz, x='P_POR_L', y='DIV_YIELD',
                 hover_data=['AGG_DIV_YIELD','PAPEL','COTACAO_GRAHAM_SETOR','COTACAO'], color='FLAG_MENOR_GRAHAM_ACIMA_DY_SETOR')
fig.show()

In [54]:
df_viz[cols].sort_values('DIV_YIELD')

,PAPEL,SETOR,COTACAO,COTACAO_GRAHAM_SETOR,RAZAO_COTACAO_GRAHAM,DIV_YIELD,AGG_DIV_YIELD,P_POR_L,OSCILACOES_%_2023,LPA,VPA,MARG_EBIT,MARG_LIQUIDA,FLAG_MENOR_GRAHAM_SETOR,FLAG_DY_ACIMA_10,FLAG_DY_ACIMA_DY_SETOR
222,PINE3,Intermediários Financeiros,4.16,4.62,1.11,0.0,6.55,7.28,194.10,0.57,5.02,NaN,0.0,1,0,0
235,RPAD3,Intermediários Financeiros,7.76,8.31,1.07,0.0,6.55,10.80,8.68,0.72,12.83,NaN,0.0,1,0,0
246,CRIV3,Intermediários Financeiros,5.70,4.38,0.77,0.0,6.55,22.45,10.04,0.25,10.29,NaN,0.0,0,0,0
238,RPAD6,Intermediários Financeiros,7.60,8.31,1.09,0.0,6.55,10.57,13.60,0.72,12.83,NaN,0.0,1,0,0
229,BRIV3,Intermediários Financeiros,10.53,12.72,1.21,1.5,6.55,9.31,14.37,1.13,19.17,NaN,0.0,1,0,0
245,BPAC3,Intermediários Financeiros,13.26,4.71,0.36,1.9,6.55,18.15,3.72,0.73,4.06,NaN,0.0,0,0,0
241,BPAC11,Intermediários Financeiros,29.23,14.12,0.48,2.5,6.55,13.34,24.08,2.19,12.18,NaN,0.0,0,0,0
236,BPAC5,Intermediários Financeiros,7.79,4.71,0.60,3.2,6.55,10.66,42.14,0.73,4.06,NaN,0.0,0,0,0
242,BPAN4,Intermediários Financeiros,7.61,5.00,0.66,3.4,6.55,14.18,25.37,0.54,6.20,NaN,0.0,0,0,0
213,PINE4,Intermediários Financeiros,3.53,4.62,1.31,4.6,6.55,6.18,127.05,0.57,5.02,NaN,0.0,1,0,0
